In [2]:
import os, json, re, textwrap, pathlib
from typing import List, Dict
import numpy as np
import requests, httpx, asyncio
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import faiss                 # CPU build is fine for <1 M vectors

/home/vishalvaka/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2025-06-16 16:32:39.951620: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-16 16:32:39.970867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750116759.995516    1662 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750116760.001985

In [3]:
OLLAMA_URL   = "http://localhost:11434"
LLM_MODEL    = "llama3:8b-instruct-q5_K_M"          # fits 12 GB VRAM
EMBED_MODEL  = "nomic-embed-text"                   # pull via SentenceTxf
DATA_DIR     = pathlib.Path("../data")
ART_DIR      = pathlib.Path("../artifacts")
DATA_DIR.mkdir(exist_ok=True, parents=True)
ART_DIR.mkdir(exist_ok=True, parents=True)

In [4]:
import requests, json, pathlib, time
from xml.etree import ElementTree as ET
from bs4 import BeautifulSoup


In [5]:
ROOT          = "https://support.stripe.com"
MASTER_SITEMAP= f"{ROOT}/sitemap.xml"
HEADERS       = {
    "User-Agent":      "Mozilla/5.0 (X11; Linux x86_64)",
    "Accept-Language": "en-US,en;q=0.9",
}
PARA_MIN_WORDS = 20          # drop boiler-plate <p> shorter than this
DELAY_SEC      = 0.4         # polite delay between HTTP requests
OUT_DIR        = pathlib.Path("data/raw")
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_JSONL      = OUT_DIR / "stripe_faqs_full.jsonl"
OUT_ART_INDEX  = OUT_DIR / "stripe_article_index.json"

In [8]:
import requests, json, pathlib, time, re, xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

###############################################################################
# Configuration
###############################################################################
ROOT          = "https://support.stripe.com"
ROBOTS_TXT    = f"{ROOT}/robots.txt"
HEADERS       = {
    "User-Agent":      "Mozilla/5.0 (X11; Linux x86_64)",
    "Accept-Language": "en-US,en;q=0.9",
}
PARA_MIN_WORDS = 20
DELAY_SEC      = 0.35                         # polite, keeps total run < 5 min

OUT_DIR        = pathlib.Path("data/raw");  OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_JSONL      = OUT_DIR / "stripe_faqs_full.jsonl"
OUT_ART_INDEX  = OUT_DIR / "stripe_article_index.json"

In [10]:
def fetch(url: str) -> str:
    r = requests.get(url, headers=HEADERS, timeout=15)
    r.raise_for_status()
    return r.text

In [11]:
def get_sitemap_urls() -> list[str]:
    """Read robots.txt and return every absolute sitemap URL."""
    print("🔍  Reading robots.txt …")
    robots = fetch(ROBOTS_TXT)
    sm_urls = []
    for line in robots.splitlines():
        if line.lower().startswith("sitemap:"):
            url = line.split(":", 1)[1].strip()
            if url.startswith("/"):          # rarely happens
                url = ROOT + url
            sm_urls.append(url)
    if not sm_urls:
        raise RuntimeError("No Sitemap lines found in robots.txt – "
                           "Stripe may have changed their layout.")
    print(f"   Found {len(sm_urls)} sitemap file(s)")
    return sm_urls

In [12]:
def iter_article_urls(sm_urls: list[str]):
    """Yield every /questions/* URL from all sitemap files."""
    q_pat = re.compile(r"/questions/")
    for sm in sm_urls:
        xml = fetch(sm)
        root = ET.fromstring(xml)
        for loc in root.findall(".//{*}loc"):
            url = loc.text.strip()
            if q_pat.search(url):
                yield url

In [13]:
def crawl():
    sitemap_urls = get_sitemap_urls()
    article_urls = sorted(set(iter_article_urls(sitemap_urls)))
    print(f"   Discovered {len(article_urls)} article URLs")

    docs = []
    for i, url in enumerate(article_urls, 1):
        html = fetch(url)
        soup = BeautifulSoup(html, "html.parser")

        title_tag = soup.select_one("h1")
        title = title_tag.get_text(strip=True) if title_tag else "Untitled"

        for p in soup.find_all("p"):
            txt = p.get_text(" ", strip=True)
            if len(txt.split()) >= PARA_MIN_WORDS:
                docs.append({"url": url, "title": title, "text": txt})

        if i % 25 == 0 or i == len(article_urls):
            print(f"   {i:>3}/{len(article_urls)} pages done "
                  f"({len(docs)} paragraphs)")

        time.sleep(DELAY_SEC)

    # save outputs
    OUT_JSONL.write_text("\n".join(json.dumps(d, ensure_ascii=False) for d in docs),
                         encoding="utf-8")
    OUT_ART_INDEX.write_text(json.dumps(article_urls, indent=2), encoding="utf-8")

    print("\n✅  Finished.")
    print(f"   Pages crawled  : {len(article_urls)}")
    print(f"   Paragraphs kept: {len(docs)}")
    print(f"   JSONL saved to : {OUT_JSONL.relative_to(pathlib.Path.cwd())}")

In [14]:
crawl()

🔍  Reading robots.txt …


RuntimeError: No Sitemap lines found in robots.txt – Stripe may have changed their layout.